# Gensim and LDA: a quick tour

First, fix the verbosity of the logger. In this example we're logging only warnings, but for a better debug, uprint all the INFOs.

In [1]:
import logging
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.WARNING)
logging.root.level = logging.WARNING

Now, it's time to get some textual data. We're gonna use the 20 newsgroups dataset (more info here: http://qwone.com/~jason/20Newsgroups). As stated by its creators, it is a collection of approximately 20,000 newsgroup documents, partitioned (nearly) evenly across 20 different newsgroups.

To make things more real, we're remving email headers, footers (like signatures) and quoted messages.

In [2]:
from sklearn import datasets
news_dataset = datasets.fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

In [4]:
# A list of text document is contained in the data variable
documents = news_dataset.data

print ("In the dataset there are", len(documents), "textual documents")
print ("And this is the first one:\n", documents[0])

In the dataset there are 18846 textual documents
And this is the first one:
 

I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!




We do now have a collection of documents. Let's start with some preprocessing steps. At first, we're gonna import all the modules we need. Then, we define a word tokenizer (https://en.wikipedia.org/wiki/Tokenization_(lexical_analysis)) with stopword removal (common words like "the", "are" and "and" are excuded from the processing, since they don't have discriminative power and they just increase the processing complexity).

In [5]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

In [7]:
def tokenize(text):
    return [token for token in gensim.utils.simple_preprocess(text) if token not in gensim.parsing.preprocessing.STOPWORDS]

print ("After the tokenizer, the previous document becomes:\n", tokenize(documents[0]))

After the tokenizer, the previous document becomes:
 ['sure', 'bashers', 'pens', 'fans', 'pretty', 'confused', 'lack', 'kind', 'posts', 'recent', 'pens', 'massacre', 'devils', 'actually', 'bit', 'puzzled', 'bit', 'relieved', 'going', 'end', 'non', 'pittsburghers', 'relief', 'bit', 'praise', 'pens', 'man', 'killing', 'devils', 'worse', 'thought', 'jagr', 'showed', 'better', 'regular', 'season', 'stats', 'lot', 'fo', 'fun', 'watch', 'playoffs', 'bowman', 'let', 'jagr', 'lot', 'fun', 'couple', 'games', 'pens', 'going', 'beat', 'pulp', 'jersey', 'disappointed', 'islanders', 'lose', 'final', 'regular', 'season', 'game', 'pens', 'rule']


Next step: tokenise all the documents and build a count dictionary, that contains the count of the tokens over the complete text corpus.

In [9]:
processed_docs = [tokenize(doc) for doc in documents]
word_count_dict = gensim.corpora.Dictionary(processed_docs)
print ("In the corpus there are", len(word_count_dict), "unique tokens")

In the corpus there are 95507 unique tokens


We might want to further lower the complexity of the process, removing all the very rare tokens (the ones appearing in less than 20 documents) and the very popular ones (the ones appearing in more than 20% documents; in our case circa 4'000)

In [10]:
word_count_dict.filter_extremes(no_below=20, no_above=0.1) # word must appear >10 times, and no more than 20% documents

In [11]:
print ("After filtering, in the corpus there are only", len(word_count_dict), "unique tokens")

After filtering, in the corpus there are only 8121 unique tokens


Let's not build the bag of words representation (https://en.wikipedia.org/wiki/Bag-of-words_model) of the text documents, to create a nice vector space model (https://en.wikipedia.org/wiki/Vector_space_model). Within this methaphor, a vector lists the multiplicity of the tokens appearing in the document. The vector is indexed by the dictionary of tokens, previously built. Note that, since a restricted subset of words appears in each document, this vector is often represented in a sparse way.

In [12]:
bag_of_words_corpus = [word_count_dict.doc2bow(pdoc) for pdoc in processed_docs]

In [14]:
bow_doc1 = bag_of_words_corpus[0]

print ("Bag of words representation of the first document (tuples are composed by token_id and multiplicity):\n", bow_doc1)
print
for i in range(5):
    print ("In the document, topic_id {} (word \"{}\") appears {} time[s]".format(bow_doc1[i][0], word_count_dict[bow_doc1[i][0]], bow_doc1[i][1]))
print ("...")

Bag of words representation of the first document (tuples are composed by token_id and multiplicity):
 [(23, 1), (341, 2), (425, 1), (450, 1), (493, 1), (514, 1), (535, 1), (808, 1), (1381, 1), (1811, 2), (1978, 1), (1987, 1), (2761, 1), (2954, 3), (3146, 1), (3232, 1), (3549, 1), (3866, 5), (3973, 1), (4015, 1), (4177, 1), (4662, 2), (5046, 2), (5052, 2), (5156, 1), (5250, 1), (5263, 2), (5447, 1), (5586, 1), (5958, 2), (5968, 1), (5999, 1), (6084, 1), (6179, 1), (6202, 1), (6512, 1), (6896, 1), (7100, 1), (7231, 1), (7380, 1), (7414, 1), (7463, 1), (7476, 1), (7837, 1)]
In the document, topic_id 23 (word "final") appears 1 time[s]
In the document, topic_id 341 (word "regular") appears 2 time[s]
In the document, topic_id 425 (word "worse") appears 1 time[s]
In the document, topic_id 450 (word "posts") appears 1 time[s]
In the document, topic_id 493 (word "beat") appears 1 time[s]
...


Now, finally, the core algorithm of the analysis: LDA. Gensim offers two implementations: a monocore one, and a multicore. We use the monocore one, setting the number of topics equal to 10 (you can change it, and check the results). Try themulticore to prove the speedup!

In [15]:
# LDA mono-core
lda_model = gensim.models.LdaModel(bag_of_words_corpus, num_topics=10, id2word=word_count_dict, passes=5)

# LDA multicore (in this configuration, defaulty, uses n_cores-1)
# lda_model = gensim.models.LdaMulticore(bag_of_words_corpus, num_topics=10, id2word=word_count_dict, passes=5)

Here's a list of the words (and their relative weights) for each topic:

In [17]:
_ = lda_model.print_topics(-1)
_

[(0,
  '0.024*edu + 0.017*com + 0.016*dos + 0.013*windows + 0.010*software + 0.009*mail + 0.007*data + 0.007*ftp + 0.006*pc + 0.006*space'),
 (1,
  '0.015*file + 0.014*image + 0.009*program + 0.008*key + 0.008*files + 0.007*code + 0.007*window + 0.006*bit + 0.006*data + 0.006*available'),
 (2,
  '0.010*drive + 0.007*car + 0.007*card + 0.006*problem + 0.006*mb + 0.005*power + 0.005*disk + 0.005*work + 0.005*hard + 0.005*scsi'),
 (3,
  '0.014*game + 0.012*team + 0.010*year + 0.009*games + 0.006*play + 0.006*season + 0.006*hockey + 0.006*st + 0.005*league + 0.005*players'),
 (4,
  '0.008*space + 0.007*information + 0.006*book + 0.006*research + 0.005*list + 0.005*earth + 0.005*mail + 0.004*university + 0.004*books + 0.004*number'),
 (5,
  '0.029*god + 0.008*jesus + 0.008*believe + 0.007*bible + 0.007*church + 0.006*christian + 0.006*christ + 0.005*religion + 0.005*word + 0.005*christians'),
 (6,
  '0.502*ax + 0.038*max + 0.008*pl + 0.006*di + 0.005*tm + 0.005*ei + 0.005*wm + 0.004*mk + 0.

Let's print now the topics composition, and their scores, for the first document. You will see that only few topics are represented; the others have a nil score.

In [18]:
for index, score in sorted(lda_model[bag_of_words_corpus[0]], key=lambda tup: -1*tup[1]):
    print ("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 10)))

Score: 0.8918003347989151	 Topic: 0.014*game + 0.012*team + 0.010*year + 0.009*games + 0.006*play + 0.006*season + 0.006*hockey + 0.006*st + 0.005*league + 0.005*players
Score: 0.058134711534788046	 Topic: 0.010*israel + 0.009*jews + 0.008*war + 0.008*armenian + 0.007*armenians + 0.007*turkish + 0.006*said + 0.006*world + 0.005*israeli + 0.005*turkey
Score: 0.03799283096267246	 Topic: 0.006*said + 0.006*years + 0.005*going + 0.004*day + 0.004*long + 0.004*things + 0.004*little + 0.004*ll + 0.004*maybe + 0.004*probably


That's wonderful! LDA is able to understand that the article is about a team game, hockey, even though the work hockey *never* appears in the document. Checking the ground truth for that document (the newsgroup category) it's actually correct! It was posted in sport/hockey category. Other topics, if any, account for less than 5%, so they have to be considered marginals (dirt).

In [19]:
news_dataset.target_names[news_dataset.target[0]]

'rec.sport.hockey'

So far, we have dealt with documents contained in the training set. What if we need to process an unseed document? Fortunately, we don't need to re-train the system (wasting lots of time), as we can just infer its topics.

In [22]:
unseen_document = "In my spare time I either play badmington or drive my car"
print ("The unseen document is composed by the following text:", unseen_document)
print

bow_vector = word_count_dict.doc2bow(tokenize(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print ("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

The unseen document is composed by the following text: In my spare time I either play badmington or drive my car
Score: 0.6350347340119411	 Topic: 0.010*drive + 0.007*car + 0.007*card + 0.006*problem + 0.006*mb
Score: 0.20496338560082367	 Topic: 0.014*game + 0.012*team + 0.010*year + 0.009*games + 0.006*play
Score: 0.020000636871351535	 Topic: 0.006*said + 0.006*years + 0.005*going + 0.004*day + 0.004*long
Score: 0.02000037465176586	 Topic: 0.008*government + 0.006*law + 0.005*mr + 0.005*public + 0.005*gun
Score: 0.02000034610222013	 Topic: 0.024*edu + 0.017*com + 0.016*dos + 0.013*windows + 0.010*software
Score: 0.02000032246553001	 Topic: 0.029*god + 0.008*jesus + 0.008*believe + 0.007*bible + 0.007*church
Score: 0.020000200286856618	 Topic: 0.010*israel + 0.009*jews + 0.008*war + 0.008*armenian + 0.007*armenians
Score: 0.02000000000540921	 Topic: 0.008*space + 0.007*information + 0.006*book + 0.006*research + 0.005*list
Score: 0.020000000002613573	 Topic: 0.502*ax + 0.038*max + 0.00

In [23]:
print ("Log perplexity of the model is", lda_model.log_perplexity(bag_of_words_corpus))

Log perplexity of the model is -7.62681524056
